# robots.txt

3 zasady grzeczności:
- stosujemy się do robots.txt
- nie dorzynamy strony
- przedstawiamy się

Dodatkowo ograniczamy głębokość i czas.
        

# Scraper

In [1]:
import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class GumtreeApartmentsSpider(scrapy.Spider):
    name = 'gumtreeapartmentsspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-'+str(i)+'/v1c9073l3200001p'+str(i)  for i in range(2,4)
        ] #ściągamy strony od drugiej do czwartej, ograniczamy głębokość
    start_urls.append(
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1' #dokładamy pierwszą stronę do odwiedzenia, która ma inny format niż pozostałe
    )
    found_apartments = []
   
    custom_settings = {
        'DOWNLOAD_DELAY': '4.0', #obejście Fail2Ban
        'ROBOTSTXT_OBEY': True #stosowanie się do poleceń z robots.txt, dzięki temu nie musimy samodzielnie ich pisać
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"})
        item_urls = []
        next_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"})
            for h in hrefs:
                item_urls.append(self.top_url + h["href"]) #dopisuje 'https://www.gumtree.pl', bo otrzymany adres jest względny, przygotowuje listę wszystkich linków
            
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item)
        
    def parse_item(self, response): #item_url - odwiedzanie strony, #self.parse_item - przetworzenie przy pomocy funkcji
        #Courtesy of Mr Sebastian Muraszewski
        found_apartments = []
        soup = BeautifulSoup(response.text, 'html.parser') 
        apartments = soup.find('div', {'class': 'vip-header-and-details'})
        apartment_details = dict()
        apartment_details['Nazwa ogłoszenia'] = apartments.find('span', class_ = 'myAdTitle').text
        apartment_details['Cena'] = apartments.find('span', class_ = 'amount').text.replace("\xa0"," ") #replace w celu usunięcia twardej spacji
        container = soup.find('ul', class_ = 'selMenu') #zebranie informacji z ramki do kontenera
        
        nazwy = container.findAll('span', class_ = 'name')
        szczegoly = container.findAll('span', class_ = 'value')
        apartment_details.update({name.text: value.text.strip() for name, value in zip(nazwy, szczegoly)}) #dodawanie elementów z ramki do listy i usunięcie whitespace'ów
        
        #zapis
        with open('plik.csv', 'a') as f: #otwieramy plik na dysku o nazwie plik.csv, plik musi być otwarty w trybie dopisywania, dlatego podajemy 'a' na końcu. Jeśli nie byłoby żadnego parametru na końcu, to plik wyłącznie do zapisu, nie możemy do niego zapisywać.
            v = apartment_details.values()
            f.write('\t'.join(v)) #zapisujemy wartości, odseparowując je od siebie separatorami, join złącza elementy

            

W komputerach jest za mało deskryptorów plikowych - ilość razy, w jakim można otworzyć plik podczas działania jednego programu/
Otwarcie pliku jest zasobem rzadkim.
Tak samo jak połączenia sieciowego i bazy danych.
Jeśli otwieramy plik, to potem trzeba go zamknąć.
Każdy błąd powoduje otwarcie pliku i jego niezamknięcie.
Dlatego używamy słówka with, które gwarantuje, że kontekst w tym bloku jest zamknięty.
Powinniśmy używać tej konstrukcji do każdej operacji z plikami, żeby zwalniać zasoby.

In [2]:
process = CrawlerProcess({
    'USER_AGENT': 'Testing MyCrawler (michal.korzycki@gmail.com)' #przedstawiamy się, ale w domu zmieniamy
})
process.crawl(GumtreeApartmentsSpider)
process.start()

2019-04-17 19:34:22 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-17 19:34:22 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.4.0-17134-Microsoft-x86_64-with-Ubuntu-18.04-bionic
2019-04-17 19:34:22 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': '4.0', 'ROBOTSTXT_OBEY': True, 'USER_AGENT': 'Testing MyCrawler (michal.korzycki@gmail.com)'}
2019-04-17 19:34:22 [scrapy.extensions.telnet] INFO: Telnet Password: e4094e8f603a7d6f
2019-04-17 19:34:22 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-04-17 19:34:22 [scrapy.middleware] INFO: Enabled downloader middlewares:

2019-04-17 19:35:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/mieszkanie-+-warszawa-srodmiescie-al-jana-pawla-ii/1004785943540911115501509> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-2/v1c9073l3200001p2)
2019-04-17 19:35:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ursynow/ursynow-3-pokoje-63m2-metro/1004786203280912391480009> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-2/v1c9073l3200001p2)
2019-04-17 19:35:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/dwupokojowe-mieszkanie-z-2006-roku-2-3p-ul-wyspowa/1004785738900910468692909> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-3/v1c9073l3200001p3)
2019-04-17 19:36:03 [scrapy.core.engine] DEBUG: Crawled

2019-04-17 19:37:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/zoliborz/kawalerka-na-zoliborzu-blisko-metra/1004785943440910468692909> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-3/v1c9073l3200001p3)
2019-04-17 19:37:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/komfortowe-3-+pokojowe-praga-poludnie-68m2-sprzedam-lub-zamienie/1004786079200910943226009> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-3/v1c9073l3200001p3)
2019-04-17 19:38:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/piekne-3+pok-saska-kepa-sprzedam-zamienie/1004786011240910943226009> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-3/v1c9073l3200001p3)
2019-04-17 19:38:06 [scrapy.core

2019-04-17 19:39:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/mieszkanie-+-warszawa-mokotow-bukowinska/1004786045380911115501509> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-2/v1c9073l3200001p2)
2019-04-17 19:40:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/mieszkanie-+-warszawa-mokotow-rajska/1004786053270911115501509> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-2/v1c9073l3200001p2)
2019-04-17 19:40:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/piaseczno/nowe-3-pok-69m-w-jozefoslawiu-garaz-komorka/1004179439440911057886109> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-2/v1c9073l3200001p2)
2019-04-17 19:40:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET

Scheduler - airBNB 
airflow.apache.org
System do schedulowania zadań

In [ ]:
W domu w Excelu mamy poprawić nazwy kolumn, przygotować plik do analizy.
Oprócz tego zamieniamy część wartości tekstowych na liczby.

In [ ]:
Otwieramy csv w Excelu za pomocą Dane/Pobieranie Danych/Z pliku tekstowego CSV